# rmANOVA on agreement values

> **Analysis type**: Single γ (calculations)

In [1]:
import json
from itertools import combinations_with_replacement
from os.path import join

import numpy as np
import pandas as pd
from dn_utils.misc import tidy_data
from dn_utils.path import path
from statsmodels.stats.anova import AnovaRM
from tqdm.notebook import tqdm

In [2]:
atlas = "combined_roi_4and5"
gamma = 2

In [3]:
# Create additional paths
gamma_str = str(float(gamma)).replace('.', '_')
path_corrmats = join(path["bsc"], "corrmats")
path_corrmats_unthr = join(path_corrmats, atlas, "unthr")
path_corrmats_unthr_gamma = join(path_corrmats_unthr, f"gamma_{gamma_str}")

# Load subject exclusion
df_exclusion = pd.read_csv(join(path["nistats"], "exclusion/exclusion.csv"), 
                           index_col=0)
ok_index = df_exclusion["ok_all"]

# Meta information about corrmats dimensions
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    corrmats_meta = json.loads(f.read()) 

# Load graph measures
d = np.load(join(path_corrmats_unthr_gamma, "d_networks.npy"))    
d_null = np.load(join(path_corrmats_unthr_gamma, "d_networks_null.npy"))

d = d[ok_index]
d_null = d_null[:, ok_index]

n_reps = len(d_null)
n_subjects = len(corrmats_meta["dim1"])
n_subjects_ok, n_conditions, n_perr_sign, n_networks, _ = d.shape

tidy_labels = [
    pd.Series(corrmats_meta["dim1"])[ok_index].to_list(),
    corrmats_meta["dim2"],
    corrmats_meta["dim3"]
]
tidy_columns = ["sub", "con", "perr_sign"]

In [4]:
pvals = np.zeros((3, n_networks, n_networks))
fstats = np.zeros((3, n_networks, n_networks))

# Real data statistics
for i, j in combinations_with_replacement(range(n_networks), r=2):
    data = tidy_data(d[:, :, :, i, j], tidy_labels, columns=tidy_columns)
    res = AnovaRM(
        data, 
        "depvar", 
        subject="sub", 
        within=["con", "perr_sign"]
    ).fit()
    
    # Store F statistic and p-values
    fstats[:, i, j] = res.anova_table["F Value"]
    pvals[:, i, j] = res.anova_table["Pr > F"]

np.save(join(path_corrmats_unthr_gamma, "d_fstats.npy"), fstats)
np.save(join(path_corrmats_unthr_gamma, "d_pvals.npy"), pvals)

In [5]:
fstats_null = np.zeros((n_reps, 3, n_networks, n_networks))

# Null distribution for F-values
for rep in tqdm(range(n_reps)):
    for i, j in combinations_with_replacement(range(n_networks), r=2):
        data = tidy_data(d_null[rep][:, :, :, i, j], tidy_labels, columns=tidy_columns)
        res = AnovaRM(
            data, 
            "depvar", 
            subject="sub", 
            within=["con", "perr_sign"]
        ).fit()
        
        fstats_null[rep, :, i, j] = res.anova_table["F Value"]
        
np.save(join(path_corrmats_unthr_gamma, "d_fstats_null.npy"), fstats_null)

  0%|          | 0/10000 [00:00<?, ?it/s]